# Do the same clusters go with another objective?

Let's do everything the same way than before, but use different objective function

In [69]:
 %matplotlib inline
import seaborn
import matplotlib.pyplot as plt
from kmeans import kmeans, randomsample
import numpy as np
import pandas as pd
import random
import os
from BorealWeights import BorealWeightedProblem
from pyomo.opt import SolverFactory
from gradutil import *
seed = 2

In [2]:
%%time
random.seed(seed)
np.random.seed(seed)
data_dir = os.path.join(os.getcwd(), '../boreal_data')
carbon = pd.read_csv(os.path.join(data_dir, 'Carbon_storage.csv'))
X = carbon.values
X[carbon.isnull()] = np.nanmin(carbon) - np.nanmax(carbon)
randomcenters = randomsample(X, 50)
centers, xtoc, dist = kmeans(X,
                             randomcenters,
                             delta=.00001,
                             maxiter=100,
                             metric='cosine',
                             verbose=0)

In [4]:
%%time
C = centers.copy()
weights = np.array([sum(xtoc==i) for i in range(len(C))])

Just noticed that I must check if the centers are averages of all the units in the cluster. I am not sure how the cosine distance affects that.

In [48]:

avs = np.array([np.array(X[xtoc==ind].mean(axis=0)) for ind in range(len(C))])

In [61]:
# C == avs
C.round() == avs.round()

I would say this is a bit weird again. I would have assumed that all are either True or False, not mixed up.

### Check this also! Is there something wrong with k-means?

Let's still pretend everything is ok and proceed to the test we planned.

In [66]:
data_dir = os.path.join(os.getcwd(), '../boreal_data')
revenues = pd.read_csv(os.path.join(data_dir, 'Timber_revenues.csv'))
Xr = revenues.values
Xr[revenues.isnull()] = np.nanmin(revenues) - np.nanmax(revenues)

In [67]:
Cr = np.array([np.array(Xr[xtoc==ind].mean(axis=0)) for ind in range(len(C))])

In [68]:
%%time
ClustProblemRev = BorealWeightedProblem(Cr,weights)
opt = SolverFactory('glpk')
resRev = opt.solve(ClustProblemRev.model, False)

In [72]:
rev_list = res_to_list(ClustProblemRev.model)
rev_list

In [73]:
optim_result_surrogate_rev = sum([Cr[ind,int(rev_list[ind])]*weights[ind] for ind in range(len(rev_list))])
optim_result_surrogate_rev

In [74]:
optim_result_surrogate_origin_rev = sum([sum(Xr[xtoc==ind][:,int(rev_list[ind])]) for ind in range(len(rev_list))])
optim_result_surrogate_origin_rev

As we can see, there is no difference at all, because we generated the results the same way. Actually the question would be, why there was difference in the carbon case?

Again comparison to original:

In [75]:
%%time
orig_weights = np.ones(len(X))
OrigProblemRev = BorealWeightedProblem(Xr,orig_weights)
opt.solve(OrigProblemRev.model, False)

In [76]:
orig_res_values_rev = res_to_list(OrigProblemRev.model)
optim_result_orig_rev = sum([Xr[ind,int(orig_res_values_rev[ind])] for ind in range(len(orig_res_values_rev))])
optim_result_orig_rev

In [77]:
(optim_result_orig_rev - optim_result_surrogate_origin_rev) / optim_result_orig_rev

This time the difference is 1.3% which really is 4x bigger than with clustering build for the same objective. Still I don't consider this extremely alarming. This aspect is one great topic to be examined.